As the model is BERT-like, we’ll train it on a task of *Masked language modeling*, i.e. the predict how to fill arbitrary tokens that we randomly mask in the dataset. This is taken care of by the example script.

In [44]:
%run -r params.py

In [30]:
from transformers import RobertaConfig
config = RobertaConfig(
    vocab_size=4096, #52_000,
    max_position_embeddings=400, #514,
    num_attention_heads=6, #12,
    num_hidden_layers=3, #6,
    type_vocab_size=1,
)

In [31]:
#Now let's re-create our tokenizer in transformers
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_dir, max_len=512)

In [32]:
!pip3 install datasets -qqq
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling
paths = ['../data/04-19-2021-train-sparql.txt', '../data/04-19-2021-train-sparql.txt']
# using load_dataset to lazy load data
dataset = load_dataset("text", data_files=paths) #text defines the type

Using custom data configuration default-ea745edc231b9cb5
Reusing dataset text (/home/shyaz/.cache/huggingface/datasets/text/default-ea745edc231b9cb5/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)


In [33]:
def encode(lines): return tokenizer(lines['text'], add_special_tokens=True, truncation=True, max_length=400)

In [34]:
dataset.set_transform(encode)
dataset = dataset['train']

In [35]:
#This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on.
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15) #mlm=MaskedLM

In [36]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=400,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [37]:
# Check that PyTorch sees cuda
import torch
torch.cuda.is_available()

True

In [38]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [39]:
model.num_parameters()

45359054

In [40]:
from transformers import Trainer, TrainingArguments 

In [41]:
!mkdir {model_roberta_mlm} -p
training_args = TrainingArguments(
    output_dir=model_roberta_mlm,
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [42]:
%%time
trainer.train()

IndexError: Invalid key: 344 is out of bounds for size 0

#### 🎉 Save final model (+ tokenizer + config) to disk

In [ ]:
trainer.save_model(model_roberta_mlm)